## Computing Causal Inference ##

In [113]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import statsmodels.formula.api as smf

In [114]:
progresa= pd.read_csv('C:\\Users\\Subhiksha\\Desktop\\574\\progresa-sample.csv\\progresa-sample.csv')

1 Descriptive analysis

1.1 Summary statistics

In [115]:
progresa.columns

Index(['year', 'sex', 'indig', 'dist_sec', 'sc', 'grc', 'fam_n', 'min_dist',
       'dist_cap', 'poor', 'progresa', 'hohedu', 'hohwag', 'welfare_index',
       'hohsex', 'hohage', 'age', 'village', 'folnum', 'grc97', 'sc97'],
      dtype='object')

In [170]:
#dropping year, folnum and village columns
progresa_new = progresa.drop(['year','folnum','village'],axis=1)

#Forming dataframe for mean, standard deviation and sorting according to index names
progresa_new1 = pd.merge(progresa_new.mean().reset_index(name='Mean'), progresa_new.std().reset_index(name='Std_dev'),on=['index']).sort_values(by ='index')

countnull= []
for i in list(progresa_new1['index']):
    countnull.append(progresa[i].isna().sum())

progresa_new1['Count_nulls']= countnull

progresa_new1

,index,Mean,Std_dev,Count_nulls
13,age,11.366460,3.167744,0
16,before,0.500000,0.500003,0
7,dist_cap,147.674452,76.063134,0
2,dist_sec,2.418910,2.234109,0
5,fam_n,7.215715,2.352900,0
4,grc,3.963537,2.499063,6549
14,grc97,3.705372,2.572387,0
12,hohage,44.436717,11.620372,10
8,hohedu,2.768104,2.656106,0
11,hohsex,0.925185,0.263095,20


In [118]:
progresa_new['progresa'].unique()

array(['0', 'basal'], dtype=object)

Progresa variable contains two values, 0 and basal and is of the type object. When progresa is 0 it falls under control group and when progresa is basal it falls under treatment group. Hence it does fit with the documentation.
This coding is made binary in difference in difference estimator as it requires only binary predictors hence basal is made integer 1 and '0' is made integer 0

1.2 Differences at baseline

1

In [172]:
#Considering only poor villages for the year 1997
progresa1 = progresa[(progresa.poor=='pobre') & (progresa.year==97)] 

#Dropping variables year, folnum and village
progresa_drop = progresa1.drop(['year','folnum','village'],axis=1)

#Calculating mean values for both treatment and control
progresa_total = pd.DataFrame(pd.merge(progresa_drop[progresa_drop.progresa == 'basal'].mean().reset_index(name='Average value (Treatment villages)'), progresa_drop[progresa_drop.progresa == '0'].mean().reset_index(name='Average value (Control villages)'),on=['index']))

t_value = []
p_value = []
stat = []

#Looping over columns in df to find t value, p value for statistical significance
for i in list(progresa_total['index']):
          t_value.append(stats.ttest_ind(list(progresa_drop[progresa_drop.progresa == 'basal'][i]), list(progresa_drop[progresa_drop.progresa == '0'][i]), nan_policy='omit').statistic)
          p_value.append(stats.ttest_ind(progresa_drop[progresa_drop.progresa == 'basal'][i], progresa_drop[progresa_drop.progresa == '0'][i], nan_policy='omit').pvalue)
          if stats.ttest_ind(progresa_drop[progresa_drop.progresa == 'basal'][i], progresa_drop[progresa_drop.progresa == '0'][i], nan_policy='omit').pvalue < 0.05:
                stat.append('YES')
          else: stat.append('NO')
        


#Reference: https://github.com/vaibhavwalvekar/Econometric-Analysis-Progresa-Mexico/blob/master/Econometric%20Analysis%20-%20Progresa%20(Mexico).ipynb

C:\Users\Subhiksha\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
C:\Users\Subhiksha\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
C:\Users\Subhiksha\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1892: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


2

In [171]:
#Adding columns to dataframe        
# Using the above ccreated list to create 3 columns as needed        
progresa_total['Difference'] = t_value   
progresa_total['p-value'] = p_value
progresa_total['Statistically_Significant'] = stat    
progresa_total

,index,Average value (Treatment villages),Average value (Control villages),Difference,p-value,Statistically_Significant
0,sex,0.519317,0.505052,2.506686,1.219172e-02,YES
1,indig,0.325986,0.332207,-1.161714,2.453603e-01,NO
2,dist_sec,2.453122,2.507662,-2.100433,3.569843e-02,YES
3,sc,0.822697,0.815186,1.668745,9.517806e-02,NO
4,grc,3.531599,3.543050,-0.400196,6.890151e-01,NO
5,fam_n,7.281327,7.302469,-0.794167,4.271039e-01,NO
6,min_dist,107.152915,103.237854,8.206584,2.358312e-16,YES
7,dist_cap,150.829074,153.769730,-3.339081,8.415005e-04,YES
8,hohedu,2.663139,2.590348,2.541229,1.105093e-02,YES
9,hohwag,544.339544,573.163558,-3.594588,3.253835e-04,YES


3

If p value is less than 0.05 then the variable is defined as statistically significant. 
Difference of most variables are between -5 to +5 range hence is considerably small. We can see that there are many statistically significant variables namely : sex, dist_sec(nearest distance to a secondary school), min_dist (min distance to an urban center), dist_cap (min distance to the capital), hohedu (years of schooling of head of household), hohwag (monthly wages of head of household),hohage (age of head of household), welfare_index (welfare index used to classify poor).


4

In this question, we calculate the differences in variables before the treatment took place. This is done in order to measure the impact of the treatment with the after-treatment data. In order to make sure the effect of the treatment is significant, there should be no differences at baseline. 
If there are no differences in baseline, then randomization of data could be plausible, but here since there are many statistically significant differences, we can say that there could be some external factors apart from treatment that led to the causal relationships.

5

Due to the differences which exist proven by statistically significant variables, we can say that there are other factors attributing to the causal relationships in the data.
Hence methods like before-after estimator or cross section estimator would not be accurate enough.
We would need to implement difference in difference inorder to cancel the effect of unobserved factors.

2 Measuring Impact

2.1 Before-after estimator

1

In [40]:
#Filtering data to calculate mean of schooling rate before treatment occured i.e 1997 in the treatment villages
progresa_97_treatment = progresa[ (progresa.year==97) & (progresa.progresa=='basal') & (progresa.poor=='pobre') ] 
#Filtering data to calculate mean of schooling rate after treatment occured i.e 1998 in the treatment villages
progresa_98_treatment = progresa[ (progresa.year==98) & (progresa.progresa=='basal') & (progresa.poor=='pobre') ]

#Calculating means
mean_treatment_98 = progresa_98_treatment.mean()['sc']
mean_treatment_97 = progresa_97_treatment.mean()['sc']


0.8464791213954308
0.8226968874033842


In [130]:
#Making dataframe in order to print means of before- after estimator in tabular form
bae= pd.DataFrame( index= ['Avg enrollment rate before treatment','Avg enrollment rate after treatment'], 
                   columns= ['Treatment group'] )

bae.loc["Avg enrollment rate before treatment","Treatment group"]= mean_treatment_97
bae.loc["Avg enrollment rate after treatment","Treatment group"]= mean_treatment_98

bae


,Treatment group
Avg enrollment rate before treatment,0.822697
Avg enrollment rate after treatment,0.846479


2

In [132]:
progresa_neww= progresa
progresa_neww['before'] = (progresa.year == 97)
progresa_neww = progresa_neww.drop(['year','folnum','village'],axis=1)

#Modeling in linear regression form to estimate relationship between schooling rate and year and filtering on only poor treatment groups.
m1= smf.ols(formula= 'sc ~ before', data=  progresa_neww[(progresa_neww.poor=='pobre') & (progresa_neww.progresa=='basal')] )
r=m1.fit()
r.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     sc   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     36.84
Date:                Wed, 05 Feb 2020   Prob (F-statistic):           1.30e-09
Time:                        08:34:08   Log-Likelihood:                -15557.
No. Observations:               36175   AIC:                         3.112e+04
Df Residuals:                   36173   BIC:                         3.113e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept          0.8465      0.003    296.922      0.000       0.841       0.852
before[T.True]    -0.0238      0.004     -6.069      0.000      -0.031      -0.016
==============================================================================
Omnibus:                    10294.733   Durbin-Watson:                   1.394
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            21582.139
Skew:                          -1.791   Prob(JB):                         0.00
Kurtosis:                       4.217   Cond. No.                         2.69
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

3

In [133]:
#Modeling in multiple variable regression form to estimate relationship between schooling rate and 6 other variables with year and filtering on only poor treatment groups.
m2= smf.ols(formula='sc ~ sex + fam_n + min_dist + progresa + hohedu + hohwag + before' , data=  progresa_neww[(progresa_neww.poor=='pobre') & (progresa_neww.progresa=='basal')])
r = m2.fit()
r.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     sc   R-squared:                       0.019
Model:                            OLS   Adj. R-squared:                  0.019
Method:                 Least Squares   F-statistic:                     119.8
Date:                Wed, 05 Feb 2020   Prob (F-statistic):          1.87e-150
Time:                        08:34:16   Log-Likelihood:                -15214.
No. Observations:               36163   AIC:                         3.044e+04
Df Residuals:                   36156   BIC:                         3.050e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept          0.7325      0.009     77.705      0.000       0.714       0.751
before[T.True]    -0.0236      0.004     -6.087      0.000      -0.031      -0.016
sex                0.0276      0.004      7.107      0.000       0.020       0.035
fam_n             -0.0017      0.001     -1.971      0.049      -0.003   -9.45e-06
min_dist           0.0006   4.77e-05     13.149      0.000       0.001       0.001
hohedu             0.0156      0.001     20.117      0.000       0.014       0.017
hohwag          3.479e-06    2.8e-06      1.241      0.215   -2.02e-06    8.98e-06
==============================================================================
Omnibus:                     9881.783   Durbin-Watson:                   1.412
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            20169.670
Skew:                          -1.737   Prob(JB):                         0.00
Kurtosis:                       4.149   Cond. No.                     4.38e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.38e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

4

In the first estimator, we only calculate averages of schooling rate before treatment took place which is 0.822697 and avg enrollment rate after treatment is 0.846479. There seems to be slight increase in the schooling rate after treatment.
In simple linear regression, we get the same results as intercept value is 0.8465 which is average schooling rate after treatment and year coefficient is -0.0238 which indicate the 2.4% increase in schooling rate due to the treatment.
In multiple regression, I have considered 6 other variables in addition to year variable namely, sex, fam_n (family size), min_dist(min distance to an urban center), hohedu(years of schooling of head of household) and hohwag (monthly wages of head of household).
Except fam_n and hohwag, all these variables have p value < 0.05 hence statistically significant. We now see a decrease in intercept which is now 0.7325 but average increase in schooling rate is the same as year coefficient is 0.0236 hence 2.4% increase.
This shows that the addition of controls did not really affect the year variable in the relationship even though average schooling rate after treatment decreased.

2.2 Cross-sectional estimator

1

In [134]:
#Filtering data to consider only poor villages after treatment for both treatment and control groups
progresa_98_treatment = progresa[(progresa.poor=='pobre') & (progresa.year==98) & (progresa.progresa=='basal')] 
progresa_98_control = progresa[(progresa.poor=='pobre') & (progresa.year==98) & (progresa.progresa=='0')] 

#Calculating means
mean_98_treatment = progresa_98_treatment.mean()['sc']
mean_98_control = progresa_98_control.mean()['sc']

In [135]:
#Making dataframe in order to print means of cross section estimator in tabular form
cse= pd.DataFrame( index= ['Avg enrollment rate after treatment'], 
                   columns= ['Treatment group', 'Control group'] )

cse.loc["Avg enrollment rate after treatment","Treatment group"]= mean_98_treatment
cse.loc["Avg enrollment rate after treatment","Control group"]= mean_98_control

cse


,Treatment group,Control group
Avg enrollment rate after treatment,0.846479,0.807637


2

In [136]:
#Changing encoding of progresa variable to 1 for treatment groups and 0 for control groups
progresa_neww.loc[progresa_neww.progresa == 'basal', 'progresa'] = 1
progresa_neww.loc[progresa_neww.progresa == '0', 'progresa'] = 0


In [84]:
#Building linear regression model in order to estimate relationship between schooling rate and progresa for after- treatment period
m1 = smf.ols(formula = 'sc ~ progresa', data=progresa_neww[(~progresa_neww.before) &(progresa_neww.poor=='pobre')] )
r = m1.fit()
r.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     sc   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                     69.87
Date:                Tue, 04 Feb 2020   Prob (F-statistic):           6.64e-17
Time:                        22:06:24   Log-Likelihood:                -11926.
No. Observations:               27450   AIC:                         2.386e+04
Df Residuals:                   27448   BIC:                         2.387e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.8076      0.004    220.676      0.000       0.800       0.815
progresa       0.0388      0.005      8.359      0.000       0.030       0.048
==============================================================================
Omnibus:                     7638.939   Durbin-Watson:                   1.734
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            15767.534
Skew:                          -1.767   Prob(JB):                         0.00
Kurtosis:                       4.140   Cond. No.                         3.01
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [137]:
#Building multiple regression model in order to estimate relationship between schooling rate and 5 other variables with progresa for after- treatment period

m2= smf.ols(formula='sc ~ progresa + sex + fam_n + min_dist + progresa + hohedu + hohwag' , data=  progresa_neww[(~progresa_neww.before) &(progresa_neww.poor=='pobre')])
r = m2.fit()
r.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     sc   R-squared:                       0.022
Model:                            OLS   Adj. R-squared:                  0.022
Method:                 Least Squares   F-statistic:                     103.4
Date:                Wed, 05 Feb 2020   Prob (F-statistic):          2.46e-129
Time:                        08:55:24   Log-Likelihood:                -11648.
No. Observations:               27440   AIC:                         2.331e+04
Df Residuals:                   27433   BIC:                         2.337e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.7019      0.011     65.169      0.000       0.681       0.723
progresa       0.0347      0.005      7.521      0.000       0.026       0.044
sex            0.0248      0.004      5.551      0.000       0.016       0.034
fam_n         -0.0024      0.001     -2.455      0.014      -0.004      -0.000
min_dist       0.0006   5.34e-05     12.131      0.000       0.001       0.001
hohedu         0.0162      0.001     17.950      0.000       0.014       0.018
hohwag      1.448e-06   3.21e-06      0.451      0.652   -4.85e-06    7.74e-06
==============================================================================
Omnibus:                     7313.877   Durbin-Watson:                   1.753
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            14684.398
Skew:                          -1.710   Prob(JB):                         0.00
Kurtosis:                       4.072   Cond. No.                     4.40e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.4e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

4

In [ ]:
In the first estimator, we only calculate averages of schooling rate after treatment took place in treatment groups which is 0.846479  and avg enrollment rate after treatment in control groups is 0.807637. There seems to be slight increase in the schooling rate after treatment.
In simple linear regression, we get the same results as intercept value is 0.8076 which is average schooling rate after treatment for control groups and progresa coefficient is 0.0388 which indicate the approximately 4% increase in schooling rate due to the treatment.
In multiple regression, I have considered 5 other variables in addition to progresa variable namely, sex, fam_n (family size), min_dist(min distance to an urban center), hohedu(years of schooling of head of household) and hohwag (monthly wages of head of household).
All these variables have p value < 0.05 hence statistically significant.
We now see a decrease in intercept which is now 0.7019 but average increase in schooling rate is little lesser ie 0.0347 hence 3.4% increase.
This shows that the addition of controls did affect the progresa variable in the relationship by a slight amount.

2.3 Differences-in-differences estimator

1

In [98]:
progresa_97_treatment = progresa[(progresa.poor=='pobre') & (progresa.year==97) & (progresa.progresa=='basal')] 
progresa_97_control = progresa[(progresa.poor=='pobre') & (progresa.year==97) & (progresa.progresa=='0')] 

progresa_98_treatment = progresa[(progresa.poor=='pobre') & (progresa.year==98) & (progresa.progresa=='basal')] 
progresa_98_control = progresa[(progresa.poor=='pobre') & (progresa.year==98) & (progresa.progresa=='0')] 

#Calculating means
mean_98_treatment = progresa_98_treatment.mean()['sc']
mean_97_treatment = progresa_97_treatment.mean()['sc']
mean_diff_treatment = mean_98_treatment - mean_97_treatment
print(mean_diff_treatment)
mean_control_98 = progresa_98_control.mean()['sc']
mean_control_97 = progresa_97_control.mean()['sc']
mean_diff_control = mean_control_98 - mean_control_97
print(mean_diff_control)

difference_in_difference = mean_diff_treatment- mean_diff_control
print(difference_in_difference)

0.023782233992046597
-0.007549046327276487
0.031331280319323085


In [139]:
#Making dataframe in order to print means of cross section estimator in tabular form
did= pd.DataFrame( index= ['Avg enrollment rate after treatment', 'Avg enrollment rate before treatment','Difference in Difference'], 
                   columns= ['Treatment group', 'Control group', 'Difference in Difference'] )

did.loc["Avg enrollment rate after treatment","Treatment group"]= mean_98_treatment
did.loc["Avg enrollment rate before treatment","Treatment group"]= mean_97_treatment
did.loc["Difference in Difference", "Treatment group"]= mean_diff_treatment

did.loc["Avg enrollment rate after treatment","Control group"]= mean_control_98
did.loc["Avg enrollment rate before treatment","Control group"]= mean_control_97
did.loc["Difference in Difference", "Control group"]= mean_diff_control

did.loc["Difference in Difference","Difference in Difference"]= difference_in_difference

did

,Treatment group,Control group,Difference in Difference
Avg enrollment rate after treatment,0.846479,0.807637,NaN
Avg enrollment rate before treatment,0.822697,0.815186,NaN
Difference in Difference,0.0237822,-0.00754905,0.0313313


In [146]:
#Converting control variables poor and year to binary form

#Converting poor to a  binary variable form
progresa_neww.loc[progresa_neww['poor'] == 'pobre', 'poor'] = 1
progresa_neww.loc[progresa_neww['poor'] ==  'no pobre', 'poor'] = 0
progresa_neww['year']= progresa['year']
#Creating a binary variable for time
progresa_neww.loc[progresa_neww.year == 97, 'time'] = 0
progresa_neww.loc[progresa_neww.year == 98, 'time'] = 1


In [150]:
#Regression with progresa and time interaction and 6 other control variables
lm_fit3 = smf.ols(formula='sc ~ progresa*time ', data=progresa_neww[(progresa_neww.poor==1)]).fit()

#Summary
lm_fit3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     sc   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     28.31
Date:                Wed, 05 Feb 2020   Prob (F-statistic):           2.76e-18
Time:                        09:31:56   Log-Likelihood:                -26242.
No. Observations:               58372   AIC:                         5.249e+04
Df Residuals:                   58368   BIC:                         5.253e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept         0.8152      0.003    233.182      0.000       0.808       0.822
progresa          0.0075      0.004      1.691      0.091      -0.001       0.016
time             -0.0075      0.005     -1.480      0.139      -0.018       0.002
progresa:time     0.0313      0.006      4.835      0.000       0.019       0.044
==============================================================================
Omnibus:                    15346.988   Durbin-Watson:                   1.397
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            30608.651
Skew:                          -1.711   Prob(JB):                         0.00
Kurtosis:                       3.937   Cond. No.                         7.67
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [152]:
#Multiple linear regression with progresa and time interaction and 6 other control variables
lm_fit4 = smf.ols(formula='sc ~ progresa*time + sex + fam_n + min_dist + progresa + hohedu + hohwag',
                  data=progresa_neww[(progresa_neww.poor==1)]).fit()

#Summary
lm_fit4.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     sc   R-squared:                       0.020
Model:                            OLS   Adj. R-squared:                  0.020
Method:                 Least Squares   F-statistic:                     148.4
Date:                Wed, 05 Feb 2020   Prob (F-statistic):          1.79e-248
Time:                        09:33:44   Log-Likelihood:                -25685.
No. Observations:               58352   AIC:                         5.139e+04
Df Residuals:                   58343   BIC:                         5.147e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept         0.7122      0.008     91.031      0.000       0.697       0.728
progresa          0.0033      0.004      0.754      0.451      -0.005       0.012
time             -0.0078      0.005     -1.548      0.122      -0.018       0.002
progresa:time     0.0315      0.006      4.901      0.000       0.019       0.044
sex               0.0285      0.003      9.138      0.000       0.022       0.035
fam_n            -0.0027      0.001     -3.942      0.000      -0.004      -0.001
min_dist          0.0006   3.73e-05     16.846      0.000       0.001       0.001
hohedu            0.0157      0.001     24.987      0.000       0.014       0.017
hohwag         2.847e-06   2.22e-06      1.280      0.200   -1.51e-06    7.21e-06
==============================================================================
Omnibus:                    14727.175   Durbin-Watson:                   1.414
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            28653.569
Skew:                          -1.659   Prob(JB):                         0.00
Kurtosis:                       3.882   Cond. No.                     5.40e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.4e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

4

In the first estimator, we have obtained a difference in difference estimator of 0.03133 which is lesser than the above two cases of before-after estimator and cross-section estimator. 

According to difference in difference approach, control variables must be in binary form, hence we have changed variables poor, progressa and year to binary form.

In the linear regression model, we have introduced year and progresa interaction variable. This interaction term has a coefficient of 0.0313 which is same as above simple difference in differences estimator. This indicates that there is 3.1% increase in schooling rates for those treatment villages after 1998. Year and progresa and no more significant variables and interaction term is significant now.

In multiple regression, interaction term has slightly increased to 0.0315. Progresa, time and hohwag are not significant variables whereas sex, family size, min dist to urban centre and education of head of family are still significant along with interaction term. This indicates that there is 3.15% increase in schooling rates for those treatment villages after 1998.


2.4 Compare the estimators

1

Counterfactual assumptions:

For before-after estimator- 
This estimator assumes that there wouldnt have been a change in schooling rate before and after the treatment in treatment group villages if there wasnt any treatment and that enrollment rate would remain the same.

For cross-section estimator-
This estimator assumes that there wouldnt have been a change in schooling rate between treated and control group villages if there wasnt any treatment and that enrollment rate would remain the same.

For difference in difference estimator- 
This estimator assumes that there wouldnt have been a change in schooling rate between treated and control group villages and before and after treatment if there wasnt any treatment and that enrollment rate would remain the same.

2

For before-after estimator: 0.0238	
For cross-section estimator:  0.0388
For difference in difference estimator: 0.0313

In all three cases, there seems to be an increase in schooling rate after treatment which suggests that there is a positive effect of the treatment on schooling rates. 

Cross section estimator has the highest estimate mostly because it failed to consider many other factor like the interaction term introduced in difference in difference estimator. Also it only consider effects which took place in 1998 and ignores anything that took place in 1997.

In before after estimator, this approach neglects the effects that would've been present already or other factors apart from treatment which couldve caused a change as it considers only treatment villages hence it received a very low estimate compared to the others.

The difference in difference estimate is lower than than of cross section because it considers an interaction variable term which accounts for change in both year and group which wasnt taken into consideration for before-after estimator and cross section. Hence those two models could be less accurate. Difference in difference eliminates such an uncertainity by taking difference hence subtracting unobserved factors. Hence it is more accurate than the other two approaches. 
